# LLM Embedding

This notebook shows the embedding process of our full corpus that will be used later to retrieve documents relevant to given queries.

In [1]:
import warnings
import json
import torch

warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import normalize_embeddings, dot_score, semantic_search

## Data Loading

In [2]:
file_path = '../clean_data/clean_corpus_fr.json'

with open(file_path, 'r') as f:
        # Parse each line as an independent JSON object
        data = [json.loads(line) for line in f]
        
print(len(data), 'documents loaded.')

10676 documents loaded.


## Embedding

Start to vectorize the text by embedding them with the DistillBERT encoder.

In [3]:
# Create a list containing all text from the corpus
corpus = [document['text'] for document in data[:200]]
docids = [document['docid'] for document in data[:200]]

# Select the LLM used to perform the embeddings
# Basic model (english only): sentence-transformers/all-MiniLM-L6-v2
# Multilingual model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
# GOAT Multilingual model: BAAI/bge-large-zh-v1.5
# bert-base-nli-mean-tokens
model = SentenceTransformer('bert-base-nli-mean-tokens') 

# Encode all the text
corpus_embeddings = model.encode(
    corpus, 
    convert_to_tensor=True, 
    show_progress_bar=True
)

corpus_embeddings = normalize_embeddings(corpus_embeddings)

# Display some information to the user
print("Information about the corpus embedding:\n"
      f"Number of document embedded: {len(corpus_embeddings)}\n"
      f"Vector size of each embedding: {corpus_embeddings[0].shape}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Information about the corpus embedding:
Number of document embedded: 200
Vector size of each embedding: torch.Size([768])


In [4]:
query = "Quelles sont les informations révélées par la Société norvégienne de radiodiffusion (NRK) concernant les adresses et les numéros des employés du complexe de l'abri antiatomique norvégien ?"  # wanted document: doc-fr-4878
top_k=10

query_embedding = model.encode(query)

# Using build-in method
search_hits = semantic_search(query_embedding, corpus_embeddings, score_function=dot_score)
scores, indices = ([hit['score'] for hit in search_hits[0]][:top_k], 
                   [hit['corpus_id'] for hit in search_hits[0]][:top_k])

print(f"Query: {query}\n\n"
      f"{'#'*50}\n"
      f"Top {top_k} most similar sentences in corpus:\n"
      f"{'#'*50}\n")

for score, idx in zip(scores, indices):
        print(f"Docid: {docids[idx]}\n"
              f"Text:{corpus[idx][:500]}\n"
              f"(Score: {score:.4f})\n")

Query: Quelles sont les informations révélées par la Société norvégienne de radiodiffusion (NRK) concernant les adresses et les numéros des employés du complexe de l'abri antiatomique norvégien ?

##################################################
Top 10 most similar sentences in corpus:
##################################################

Docid: doc-fr-4878
Text:continuité gouvernement cog principe établissement procédures définies permettent gouvernement poursuivre activités essentielles cas événement catastrophique comme guerre nucléaire continuité gouvernement a élaboré gouvernement britannique avant pendant seconde guerre mondiale afin contrer menaces telles bombardements luftwaffe pendant bataille angleterre besoin plans continuité gouvernementale a gagné urgence prolifération nucléaire pendant après guerre froide pays développé tels plans afin évi
(Score: 11.0440)

Docid: doc-fr-3771
Text:éphores littéralement surveillants oráô surveiller composent directoire cinq magistrats annu